In [ ]:
# Install R
!apt-get install -y r-base

# Install rpy2 for R support
!pip install -q rpy2

# Load R magic extension
%load_ext rpy2.ipython

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
r-base is already the newest version (4.5.1-1.2204.0).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%R
# Load required libraries
library(dplyr)
library(readr)

# Set working directory to project folder
setwd("/content/drive/MyDrive/R language")

# Create Results folder if it doesn't exist
if (!dir.exists("Results")) {
  dir.create("Results")
}

# Define classification function
classify_DEG <- function(logFC, padj) {
  if (padj < 0.05 && logFC > 1) {
    return("Upregulated")
  } else if (padj < 0.05 && logFC < -1) {
    return("Downregulated")
  } else {
    return("Not Significant")
  }
}

# List of input files
input_files <- c("/content/drive/MyDrive/R language/DEGs_Data_1.csv", "/content/drive/MyDrive/R language/DEGs_Data_2.csv")

# Initialize list for summaries
summary_list <- list()

# Loop through files
for (file in input_files) {
  # Read data
  data <- read_csv(file)

  # Apply classification
  data$Classification <- mapply(classify_DEG, data$logFC, data$padj)

  # Save processed data
  output_file <- paste0("Results/processed_", basename(file))
  write_csv(data, output_file)

  # Create summary table
  summary_table <- data %>%
    group_by(Classification) %>%
    summarise(Count = n(), .groups = "drop")

  # Save summary table
  summary_file <- paste0("Results/summary_", basename(file))
  write_csv(summary_table, summary_file)

  # Store for combined summary
  summary_table$Source <- basename(file)
  summary_list[[file]] <- summary_table

  # Print summary
  print(paste("Summary for", file))
  print(summary_table)
}

# Combine summaries
combined_summary <- bind_rows(summary_list)

# Save combined summary
write_csv(combined_summary, "Results/combined_summary.csv")

Rows: 22283 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Gene_Id
dbl (2): padj, logFC

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "Summary for /content/drive/MyDrive/R language/DEGs_Data_1.csv"
# A tibble: 1 × 3
  Classification  Count Source         
  <chr>           <int> <chr>          
1 Not Significant 22283 DEGs_Data_1.csv
Rows: 54675 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Gene_Id
dbl (2): padj, logFC

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "Summary for /content/drive/MyDrive/R language/DEGs_Data_2.csv"
# A tibble: 3 × 3
  Classification  Count Source         
  <chr>           <int> <chr>          
1 Do